In [5]:
import random
from collections import defaultdict, deque
from itertools import product, chain
import copy
import time
import pdb

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import Sequential, load_model, model_from_json
from tensorflow.keras.optimizers import Adam

from IPython.display import clear_output


In [6]:
BOARD_SIZE = 3
MAX_MOVES = BOARD_SIZE * BOARD_SIZE
INIT_BOARD = ((' ',' ',' '),(' ',' ',' '),(' ',' ',' '),)

def print_board(board):
    """return string representation of board"""
    retval = ''
    for i, row in enumerate(board):
        if i:
            retval += "===========\n"
        retval += " %s\n" % " | ".join(row)
    retval += "\n"
    return retval

class Game:
    """Maintain game state"""
    
    def __init__(self, startplayer='X'):
        self.board = INIT_BOARD
        self.player = startplayer
        self.history = []

    def __repr__(self):
        """string representation of board"""
        return print_board(self.board)
        
    def play(self, move, record=True):
        """given move as row, col, player tuple, update board
        record=True: update game state and return board; 
        record=False: just return resulting board, i.e. for evaluation
        """
        i, j, player = move
        if player != self.player:
            raise(Exception("play: wrong player %s" % (player)))
        elif i >= len(self.board) or j >= len(self.board):
            raise(Exception("play: bad square coords %d, %d" % (i,j)))
        elif self.board[i][j] != ' ':
            raise(Exception("play: move to non-empty square"))
        else:
            # new tuple, same except set square = player
            new_board = tuple(row if r != i \
                              else tuple(player if c==j else square for c, square in enumerate(row)) \
                              for r, row in enumerate(self.board))
            if record:
                self.board=new_board
                self.history.append(self.board)
                self.player = 'O' if self.player=='X' else 'X'
                
        return new_board
    
    def is_winner(self, player='X'):
        b_a = np.array(self.board)
        if any(all(b_a[i, :] == player) for i in range(BOARD_SIZE)):
            return True
        if any(all(b_a[:, j] == player) for j in range(BOARD_SIZE)):
            return True
        if all(np.diagonal(b_a) == player):
            return True
        if all(np.diagonal(np.fliplr(b_a)) == player):
            return True
        # no winning conditions are True
        return False
    
g = Game()
g


   |   |  
   |   |  
   |   |  


In [7]:
g.play((1,1,'X'))
# some bad moves
#g.play((2,2,'X')) # wrong player
#g.play((1,1,'O')) # occupied square
#g.play((3,1,'O')) # off board
g.play((0,1,'O'))
g.play((0,0,'X'))
g.play((2,2,'O'))
g.play((2,0,'X'))
g.play((1,0,'O'))
g.play((0,2,'X'))
print(g.is_winner('O'))
print(g.is_winner('X'))
g

False
True


 X | O | X
 O | X |  
 X |   | O


In [8]:
# check winning boards
for bx in [((' ', ' ', ' '),(' ', ' ', ' '),(' ', ' ', ' ')),
           (('X', 'X', 'X'),(' ', ' ', ' '),(' ', ' ', ' ')),
           ((' ', ' ', ' '),('X', 'X', 'X'),(' ', ' ', ' ')),
           ((' ', ' ', ' '),(' ', ' ', ' '),('X', 'X', 'X')),
           (('X', ' ', ' '),('X', ' ', ' '),('X', ' ', ' ')),
           ((' ', 'X', ' '),(' ', 'X', ' '),(' ', 'X', ' ')),
           ((' ', ' ', 'X'),(' ', ' ', 'X'),(' ', ' ', 'X')),
           (('X', ' ', ' '),(' ', 'X', ' '),(' ', ' ', 'X')),
           ((' ', ' ', 'X'),(' ', 'X', ' '),('X', ' ', ' ')),]:
    g.board = bx
    print(g.is_winner('X'))
    print(g)


False
   |   |  
   |   |  
   |   |  


True
 X | X | X
   |   |  
   |   |  


True
   |   |  
 X | X | X
   |   |  


True
   |   |  
   |   |  
 X | X | X


True
 X |   |  
 X |   |  
 X |   |  


True
   | X |  
   | X |  
   | X |  


True
   |   | X
   |   | X
   |   | X


True
 X |   |  
   | X |  
   |   | X


True
   |   | X
   | X |  
 X |   |  




In [9]:
LEARNING_RATE = 0.25
DISCOUNT_RATE = 0.05
EXPLORATION_RATE = 0.025 

class RLagent:
    """Simple reinforcement learning agent
    initialize with an array (defaultdict) with value for each board
    select_move: given a board, determine valid moves, play move with best value 
    (or explore random move with probability EXPLORATION_RATE)
    train: traverse all game boards in game state history, 
    update value array based on winner, discount rate, learning rate
    """
    def __init__(self, 
                 game, 
                 V_dict,
                 player='O',
                 learning_rate=LEARNING_RATE,
                 discount_rate=DISCOUNT_RATE,
                 exploration_rate=EXPLORATION_RATE
                ):
        self.game = game
        self.V = V_dict
        self.player = player
        self.learning_rate = learning_rate
        self.discount_rate = discount_rate
        self.exploration_rate = exploration_rate
        
    def valid_moves(self):
        retlist = []
        for i, row in enumerate(self.game.board):
            for j, colval in enumerate(row):
                if colval == ' ':
                    move = (i,j, self.player)
                    retlist.append(move)
        return retlist

    def select_move(self, verbose=False, exploration_rate=None):
        """select best scoring action, 
        if more than one have best score pick random from best"""
        moves = self.valid_moves()

        if not exploration_rate:
            exploration_rate = self.exploration_rate
        
        # choose a random move some % of time specified by exploration rate
        if random.uniform(0,1) < exploration_rate:
            # set all scores to 0.5
            scores = [0.5 for b in moves]
            boards = [self.game.play(move, record=False) for move in moves]
            if verbose:
                print("Random exploration")
        else:
            # look up boards without recording
            boards = [self.game.play(move, record=False) for move in moves]
            # look up scores
            scores = [self.V[board] for board in boards]

        if verbose:
            for i, s in enumerate(scores):
                print("%d.  %.04f\n%s" % (i, s, print_board(boards[i])))

        # if player is X, choose highest prob of X winning else lowest prob of X winning
        best_score = max(scores) if self.player == 'X' else min(scores)
        # get all scores matching best
        best_moves = [moves[i] for i, score in enumerate(scores) if score == best_score]
        # pick one
        return random.choice(best_moves)

    def train(self):
        # update value function based on winner at end of game
        
        # last board gets value of 1 if X wins, 0 if O wins, 0.5 if draw
        reward = 1  if self.game.is_winner('X') \
            else -1 if self.game.is_winner('O') \
            else 0
        
        for b in reversed(self.game.history):
            # update value of each board you see, by (learning rate %) of the way to current reward
            old = self.V[b]
            self.V[b] = old + (reward - old) * self.learning_rate
            # discount reward as boards get older 
            reward = reward * (1-self.discount_rate)
            
            if verbose:
                print("old %.04f new %.04f\n%s"% (old, self.V[b], print_board(b)))


In [10]:
class HumanAgent:
    """human player, get moves from input"""

    def __init__(self, game, player):
        self.game = game
        self.player = player
        
    def get_dim(self, prompt):
        """get a single row or column input"""
        dim = None
        while dim not in range(1,BOARD_SIZE+1):
            print(prompt)
            inputstr = input()
            dim = int(inputstr) if inputstr else -1
        return int(dim)-1

    def get_move(self):
        while True:
            row = self.get_dim("Enter row: ")
            col = self.get_dim("Enter column: ")
            try:
                self.game.play((row, col, self.player))
            except Exception as e:
                print(str(e))
                continue
            break
        return row, col

def play_again():
    print('Play again? (y or n)')
    return input().lower().startswith('y')
    


In [7]:
# play human v. human

while True:
    g = Game()    
    playerX = HumanAgent(g, 'X')
    playerO = HumanAgent(g, 'O')
    
    max_moves = BOARD_SIZE * BOARD_SIZE
    winner = None
    
    for _ in range(max_moves):
        clear_output()
        print(g)
        
        player = g.player
        if player == 'X':
            row, col = playerX.get_move()
        else:
            row, col = playerO.get_move()

        if g.is_winner(player):
            winner = player
            break

    clear_output()
    print(g)
            
    if winner is None:
        print("Draw")
    else:
        print("%s wins!" % winner)

    if not play_again():
        print("Bye!")
        break


 X | O | X
 X | O |  
 X |   | O


X wins!
Play again? (y or n)
n
Bye!


'   |   | X\n===========\n   | X |  \n===========\n X |   |  \n\n'

In [19]:
# need print_board_as_html_table

from IPython.display import HTML
s  = '<script type="text/Javascript">'
s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
s += 'win.document.body.innerHTML = \'' + str(g).replace("\n",'<br />') + '\';'
s += '</script>'
HTML(s)


In [11]:
%%time
# play a bunch of games computer v. computer and update V table

START_EXPLORATION_RATE = 0.25
NUM_GAMES = 99999
V = defaultdict(lambda: 0)
verbose = False

def play_game(V,
              board_size=BOARD_SIZE,
              exploration_rate=START_EXPLORATION_RATE,
              verbose=verbose):

    g = Game()    
    playerX = RLagent(g, V, 'X')
    playerO = RLagent(g, V, 'O')
    
    max_moves = BOARD_SIZE * BOARD_SIZE
    winner = None

    for move_counter in range(max_moves): 
        clear_output()
        print(g)
        
        player = g.player

        if player == 'X':
            move = playerX.select_move(verbose, exploration_rate=exploration_rate)
        else:
            move = playerO.select_move(verbose, exploration_rate=exploration_rate)
        g.play(move)
        
        if g.is_winner(player):
            winner = player
            break

    if winner is None:
        print("Draw")
    else:
        print("%s wins!" % winner)

    if verbose:
        for i, b in enumerate(g.history):
            print("Move %d" % i)
            print(V[b])
            print(print_board(b))
        
    # update V
    # players share V array and g game history so we can train either player, only train once
    playerO.train()


for game_counter in range(NUM_GAMES):
    # linear epsilon decay
    exploration_rate = (1 - game_counter/NUM_GAMES) * START_EXPLORATION_RATE
    play_game(V, exploration_rate=exploration_rate)
    if game_counter % 1000 == 999:
        print("%s: %6d" % (time.strftime("%H:%M:%S"), game_counter+1))


KeyboardInterrupt: 

In [9]:
# check out a few V values

b = ((' ', ' ', ' '),(' ', ' ', ' '),(' ', ' ', ' '))
print("%f\n%s"% (V[b], print_board(b)))

b = ((' ', ' ', ' '),(' ', ' ', ' '),('X', ' ', ' '))
print("%f\n%s"% (V[b], print_board(b)))

b = ((' ', 'O', ' '),(' ', ' ', ' '),('X', ' ', ' '))
print("%f\n%s"% (V[b], print_board(b)))

b = (('X', 'O', ' '),(' ', ' ', ' '),('X', ' ', ' '))
print("%f\n%s"% (V[b], print_board(b)))

b = (('X', 'O', ' '),('O', ' ', ' '),('X', ' ', ' '))
print("%f\n%s"% (V[b], print_board(b)))

b = (('X', 'O', ' '),('O', ' ', ' '),('X', ' ', 'X'))
print("%f\n%s"% (V[b], print_board(b)))

b = (('X', 'O', ' '),('O', 'O', ' '),('X', ' ', 'X'))
print("%f\n%s"% (V[b], print_board(b)))

b = (('X', 'O', ' '),('O', 'O', ' '),('X', 'X', 'X'))
print("%f\n%s"% (V[b], print_board(b)))



0.000000
   |   |  
   |   |  
   |   |  


-0.193445
   |   |  
   |   |  
 X |   |  


0.663005
   | O |  
   |   |  
 X |   |  


0.320389
 X | O |  
   |   |  
 X |   |  


0.337251
 X | O |  
 O |   |  
 X |   |  


0.274275
 X | O |  
 O |   |  
 X |   | X


0.382644
 X | O |  
 O | O |  
 X |   | X


0.999968
 X | O |  
 O | O |  
 X | X | X




In [10]:
# play human vs. computer and learn interactively

while True:
    g = Game()    
    playerX = HumanAgent(g, 'X')
    playerO = RLagent(g, V, 'O')
    
    max_moves = BOARD_SIZE * BOARD_SIZE
    winner = None
    
    for _ in range(max_moves):
        clear_output()
        print(g)
        
        player = g.player
        if player == 'X':
            row, col = playerX.get_move()
        else:
            move = playerO.select_move(verbose=False, exploration_rate=0)            
            g.play(move)

        if g.is_winner(player):
            winner = player
            break

    clear_output()
    print(g)
    if winner is None:
        print("Draw")
    else:
        print("%s wins!" % winner)

    playerO.train()
        
    if not play_again():
        print("Bye!")
        break


 X | O | X
 O | O | X
 X | X | O


Draw
Play again? (y or n)
n
Bye!


In [13]:
### export csv

def v_to_dataframe(V):

    z = defaultdict(list)

    for s, v in V.items():
        # flatten
        s = tuple(chain.from_iterable(s))
        # map s to floats
        #s = tuple(0 if i==' ' else 1 if i=='X' else -1 for i in s)
        #templist = z[s]
        #templist.append(v)
        z[s]=v
        
    Vdf = pd.DataFrame(z.keys())
    Vdf['val']=z.values()
    return Vdf

Vdf = v_to_dataframe(V)
Vdf.to_csv('V.csv')
Vdf.head(30)


,0,1,2,3,4,5,6,7,8,val
0,X,,,,,,,,,-1.450256e-01
1,,X,,,,,,,,-1.934452e-01
2,,,X,,,,,,,-1.795083e-01
3,,,,X,,,,,,9.994197e-63
4,,,,,X,,,,,-1.934452e-01
5,,,,,,X,,,,-1.450839e-01
6,,,,,,,X,,,-1.088868e-01
7,,,,,,,,X,,-1.934452e-01
8,,,,,,,,,X,-1.043612e-01
9,O,,,,,,,,X,7.582467e-01


In [14]:
def V_from_csv(filename):
    Vdf = pd.read_csv(filename)
    Vdf = Vdf[['0', '1', '2', '3', '4', '5', '6', '7', '8', 'val']]
    z = defaultdict(lambda: 0.5)
    # make rows into tuples 
    for row in range(len(Vdf)):
        b = ((Vdf.iloc[row][0], Vdf.iloc[row][1], Vdf.iloc[row][2]),
             (Vdf.iloc[row][3], Vdf.iloc[row][4], Vdf.iloc[row][5]),
             (Vdf.iloc[row][6], Vdf.iloc[row][7], Vdf.iloc[row][8]),
            )
        z[b] = Vdf.iloc[row]['val']
    return Vdf

Vdf = V_from_csv('V.csv')
Vdf.head()

# pickle would have been easier

,0,1,2,3,4,5,6,7,8,val
0,X,,,,,,,,,-1.450256e-01
1,,X,,,,,,,,-1.934452e-01
2,,,X,,,,,,,-1.795083e-01
3,,,,X,,,,,,9.994197e-63
4,,,,,X,,,,,-1.934452e-01


In [15]:
# agent that uses neural network instead of lookup table/linear model

LEARNING_RATE = 0.4
DISCOUNT_RATE = 0.05
EXPLORATION_RATE = 0.1
QUEUE_LEN = 1000
INPUT_DIM=9

# V_hist_columns=['0','1','2','3','4','5','6','7','8','val']
# V_hist = pd.DataFrame(columns=V_hist_columns)

class DeepRLagent:
    """Instead of updating a V dict in training, add experienced reward values to dataframe
    and train neural net to predict boards based on the experienced values
    """
    
    def __init__(self, 
                 game, 
                 V_model,
                 V_hist,
                 player='O',
                 discount_rate=DISCOUNT_RATE,
                 exploration_rate=EXPLORATION_RATE
                ):
        self.game = game
        self.player = player
        self.discount_rate = discount_rate
        self.exploration_rate = exploration_rate
        self.V_hist = V_hist
        self.V_model = V_model
        self.best_metric = None
        self.best_model = None
        
    def valid_moves(self):
        retlist = []
        for i, row in enumerate(self.game.board):
            for j, colval in enumerate(row):
                if colval == ' ':
                    move = (i,j, self.player)
                    retlist.append(move)
        return retlist

    def select_move(self, verbose=False, exploration_rate=None):
        """select best scoring action, 
        if more than one have best score pick random from best"""
        moves = self.valid_moves()

        if not exploration_rate:
            exploration_rate = self.exploration_rate
        
        # choose a random move some % of time specified by exploration rate
        if random.uniform(0,1) < exploration_rate: #or not hasattr(self.V_model, 'coefs_'):
            # set all scores to 0.5
            scores = [0.5 for b in moves]
            boards = [self.game.play(move, record=False) for move in moves]
            if verbose:
                print("Random exploration")
        else:
            # look up boards without recording
            boards = [self.game.play(move, record=False) for move in moves]
            # look up scores
            flatboards = [np.array(self.flatten(b)).reshape(1,-1) for b in boards]
            scores = [self.V_model.predict(b) for b in flatboards]

        if verbose:
            for i, s in enumerate(scores):
                print("%d.  %.04f\n%s" % (i, s, print_board(boards[i])))

        # if player is X, choose highest prob of X winning else lowest prob of X winning
        best_score = max(scores) if self.player == 'X' else min(scores)
        # get all scores matching best
        best_moves = [moves[i] for i, score in enumerate(scores) if score == best_score]
        # pick one
        return random.choice(best_moves)

    def flatten(self, b):
        """convert board to a flat array of ints representation"""
        #flatten
        retlist = list(chain.from_iterable(b))
        # convert to ints
        retlist = [1 if player == 'X' else -1 if player=='O' else 0 for player in retlist]
        return retlist
    
    def train(self, initial_epoch=0, evaluate=False):
        # update value function based on winner at end of game
        
        # last board gets value of 1 if X wins, 0 if O wins, 0.5 if draw
        reward = 1  if self.game.is_winner('X') \
            else -1 if self.game.is_winner('O') \
            else 0
        
        for b in reversed(self.game.history):
            # append board, reward to queue/pandas dataframe
            # flatten
            observation = self.flatten(b)
            observation.append(reward)
            #append in place
            self.V_hist.loc[self.V_hist.shape[0]]=observation
            # discount value as boards get older 
            reward = reward * (1-self.discount_rate)
            
            if verbose:
                print("new reward %.04f\n%s"% (reward, print_board(b)))
        train_X = self.V_hist.iloc[-10000:,:9]
        train_y = self.V_hist.iloc[-10000:,-1]
        self.V_model.fit(train_X,
                         train_y,
                         batch_size=self.V_hist.shape[0], 
                         initial_epoch=0,
                         epochs=1,
                         verbose=0
                        )
        if evaluate:
            train_y_predict = self.V_model.predict(train_X)
            mse = mean_squared_error(train_y, train_y_predict)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(train_y, train_y_predict)
            print("MSE: %.4f RMSE %.4f MAE %.4f" % (mse, 
                                                    np.sqrt(mse), 
                                                    mae))            
            global best_metric, best_model
            if best_model is None or mae < best_metric:
                best_model = copy.copy(V_model)
                best_metric = mae
                save_model('model')
                    

In [16]:
def build_ols_model(input_size = INPUT_DIM, 
                    n_hidden_layers=1, 
                    largest_layer_size=32, 
                    activation='relu',
                    reg_penalty=0.0,
                    dropout=False,
                    verbose=True
                   ):

    model = Sequential()
    hidden_layer_size=largest_layer_size

    for i in range(n_hidden_layers):
        if verbose:
            print("layer %d size %d, %s, reg_penalty %.8f, dropout %.3f" % (i + 1, 
                                                                            hidden_layer_size, 
                                                                            activation,
                                                                            reg_penalty,
                                                                            dropout,
                                                                           ))
        if i and dropout:
            model.add(Dropout(dropout))

        if i==0: # first layer, specify input shape
            model.add(Dense(input_shape=(input_size,),
                            units = hidden_layer_size, 
                            activation = activation,
                            kernel_initializer = keras.initializers.glorot_uniform(),
                            kernel_regularizer=keras.regularizers.l2(reg_penalty),
                            name = "Dense%02d" % i))
        else: #use implicit input shape
            model.add(Dense(units = hidden_layer_size, 
                            activation = activation,
                            kernel_initializer = keras.initializers.glorot_uniform(),
                            kernel_regularizer=keras.regularizers.l2(reg_penalty),
                            name = "Dense%02d" % i))

        hidden_layer_size = hidden_layer_size // 2

    model.add(Dense(1, activation='linear'))

    if verbose:
        print(model.summary())

    # reduce learning rate by 1/10 vs. default, we are calling repeatedly for 1 iteration
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.0001), metrics=['mae'])

    return model

def load_model(filename, verbose=True):
    json_file = open('%s.json' % filename, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    mymodel = model_from_json(loaded_model_json)
    # load weights into new model
    mymodel.load_weights("%s.h5" % filename)
    print("Loaded saved V_model")
    mymodel.compile(loss='mse', optimizer=Adam(learning_rate=0.00001), metrics=['mae'])
    if verbose:
        print(mymodel.summary())
    return mymodel

def save_model(filename, verbose=True):
    # serialize model to JSON
    model_json = V_model.to_json()
    with open("%s.json" % filename, "w") as json_file:
        json_file.write(model_json)
        # serialize weights to HDF5
        V_model.save_weights("%s.h5" % filename)
    if verbose:
        print("Saved '%s' to disk" % filename)


In [17]:
#%%time
# use DeepRLAgent
# play a bunch of games computer v. computer and update V function approximator

START_EXPLORATION_RATE = 0.04
NUM_GAMES = 9999
verbose = False

V_hist_columns=['0','1','2','3','4','5','6','7','8','val']
V_hist = pd.DataFrame(columns=V_hist_columns)

best_metric=None
best_model=None

# V_model = build_ols_model(input_size = INPUT_DIM,
#                            n_hidden_layers=3, 
#                            largest_layer_size=128,
#                            activation='tanh',
#                            reg_penalty=0.0,
#                            dropout=0.0,
#                            verbose=True)        

# load best previous model
V_model = load_model('model')
    
def play_game(V,
              board_size=BOARD_SIZE,
              exploration_rate=START_EXPLORATION_RATE,
              train=True,
              evaluate=False,
              game_counter=0,
              verbose=verbose):

    g = Game()
    global V_model, V_hist
     
    playerX = DeepRLagent(g, V_model, V_hist, 'X')
    playerO = DeepRLagent(g, V_model, V_hist, 'O')
    
    max_moves = BOARD_SIZE * BOARD_SIZE
    winner = None
    #pdb.set_trace()
    for move_counter in range(max_moves):        
        player = g.player

        move = playerX.select_move(verbose, exploration_rate=exploration_rate) if player == 'X' \
            else playerO.select_move(verbose, exploration_rate=exploration_rate)
        g.play(move)
        
        if g.is_winner(player):
            winner = player
            break

#     if winner is None:
#         print("Draw")
#     else:
#         print("%s wins!" % winner)

    if verbose:
        for i, b in enumerate(g.history):
            print("Move %d" % i)
            print(V[b])
            print(print_board(b))
        
    # update V
    if train:
        playerO.train(initial_epoch=game_counter, evaluate=evaluate)
    
    return winner

draw_count = 0
draw_counts = []
for game_counter in range(NUM_GAMES):
    # linear epsilon decay
    exploration_rate = (1 - game_counter/NUM_GAMES) * START_EXPLORATION_RATE

    if game_counter % 100 == 0 and game_counter:
        print("%s: Finished %6d Games, Draws in last 100 games: %d" % (time.strftime("%H:%M:%S"), game_counter, draw_count))
        draw_counts.append(draw_count)
        draw_count = 0
        evaluate = True
    else:
        evaluate = False

    winner = play_game(V_model, V_hist, exploration_rate=exploration_rate, train=True, evaluate=evaluate,
                       game_counter=game_counter)
        
    if winner is None:
        draw_count += 1
        
    print("%s: Game %d %s" % (time.strftime("%H:%M:%S"), game_counter,
                              "Draw" if winner is None else "%s wins!" % winner))


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Loaded saved V_model
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense00 (Dense)              (None, 128)               1280      
_________________________________________________________________
Dense01 (Dense)              (None, 64)                8256      
_________________________________________________________________
Dense02 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total par

12:51:26: Game 248 Draw
12:51:26: Game 249 Draw
12:51:26: Game 250 X wins!
12:51:26: Game 251 Draw
12:51:26: Game 252 Draw
12:51:26: Game 253 Draw
12:51:26: Game 254 Draw
12:51:26: Game 255 Draw
12:51:27: Game 256 Draw
12:51:27: Game 257 Draw
12:51:27: Game 258 Draw
12:51:27: Game 259 Draw
12:51:27: Game 260 Draw
12:51:27: Game 261 Draw
12:51:27: Game 262 Draw
12:51:27: Game 263 X wins!
12:51:28: Game 264 Draw
12:51:28: Game 265 Draw
12:51:28: Game 266 Draw
12:51:28: Game 267 Draw
12:51:28: Game 268 Draw
12:51:28: Game 269 Draw
12:51:28: Game 270 Draw
12:51:28: Game 271 Draw
12:51:29: Game 272 Draw
12:51:29: Game 273 Draw
12:51:29: Game 274 Draw
12:51:29: Game 275 Draw
12:51:29: Game 276 Draw
12:51:29: Game 277 Draw
12:51:29: Game 278 Draw
12:51:29: Game 279 Draw
12:51:30: Game 280 Draw
12:51:30: Game 281 Draw
12:51:30: Game 282 Draw
12:51:30: Game 283 Draw
12:51:30: Game 284 Draw
12:51:30: Game 285 Draw
12:51:30: Game 286 Draw
12:51:30: Game 287 Draw
12:51:31: Game 288 Draw
12:51:31: 

12:52:13: Game 573 Draw
12:52:14: Game 574 Draw
12:52:14: Game 575 Draw
12:52:14: Game 576 Draw
12:52:14: Game 577 Draw
12:52:14: Game 578 Draw
12:52:15: Game 579 Draw
12:52:15: Game 580 Draw
12:52:15: Game 581 Draw
12:52:15: Game 582 Draw
12:52:15: Game 583 Draw
12:52:15: Game 584 X wins!
12:52:16: Game 585 Draw
12:52:16: Game 586 Draw
12:52:16: Game 587 Draw
12:52:16: Game 588 X wins!
12:52:16: Game 589 Draw
12:52:16: Game 590 Draw
12:52:16: Game 591 Draw
12:52:17: Game 592 X wins!
12:52:17: Game 593 Draw
12:52:17: Game 594 X wins!
12:52:17: Game 595 Draw
12:52:17: Game 596 Draw
12:52:17: Game 597 Draw
12:52:17: Game 598 Draw
12:52:18: Game 599 Draw
12:52:18: Finished    600 Games, Draws in last 100 games: 88
MSE: 0.0507 RMSE 0.2252 MAE 0.0904
12:52:18: Game 600 Draw
12:52:18: Game 601 X wins!
12:52:18: Game 602 Draw
12:52:18: Game 603 Draw
12:52:19: Game 604 X wins!
12:52:19: Game 605 Draw
12:52:19: Game 606 Draw
12:52:19: Game 607 Draw
12:52:19: Game 608 Draw
12:52:20: Game 609 Dra

12:53:10: Game 899 Draw
12:53:10: Finished    900 Games, Draws in last 100 games: 95
MSE: 0.0426 RMSE 0.2063 MAE 0.0811
Saved 'model' to disk
12:53:10: Game 900 Draw
12:53:10: Game 901 Draw
12:53:11: Game 902 Draw
12:53:11: Game 903 Draw
12:53:11: Game 904 Draw
12:53:11: Game 905 Draw
12:53:11: Game 906 Draw
12:53:11: Game 907 Draw
12:53:11: Game 908 Draw
12:53:12: Game 909 X wins!
12:53:12: Game 910 Draw
12:53:12: Game 911 Draw
12:53:12: Game 912 X wins!
12:53:12: Game 913 Draw
12:53:12: Game 914 Draw
12:53:12: Game 915 X wins!
12:53:13: Game 916 Draw
12:53:13: Game 917 Draw
12:53:13: Game 918 Draw
12:53:13: Game 919 Draw
12:53:13: Game 920 Draw
12:53:13: Game 921 Draw
12:53:14: Game 922 Draw
12:53:14: Game 923 Draw
12:53:14: Game 924 Draw
12:53:14: Game 925 Draw
12:53:14: Game 926 X wins!
12:53:14: Game 927 Draw
12:53:14: Game 928 Draw
12:53:15: Game 929 Draw
12:53:15: Game 930 Draw
12:53:15: Game 931 Draw
12:53:15: Game 932 Draw
12:53:15: Game 933 Draw
12:53:15: Game 934 Draw
12:53:

12:54:02: Game 1210 Draw
12:54:03: Game 1211 Draw
12:54:03: Game 1212 Draw
12:54:03: Game 1213 Draw
12:54:03: Game 1214 Draw
12:54:03: Game 1215 Draw
12:54:04: Game 1216 Draw
12:54:04: Game 1217 Draw
12:54:04: Game 1218 X wins!
12:54:04: Game 1219 Draw
12:54:04: Game 1220 Draw
12:54:04: Game 1221 Draw
12:54:05: Game 1222 Draw
12:54:05: Game 1223 Draw
12:54:05: Game 1224 X wins!
12:54:05: Game 1225 X wins!
12:54:05: Game 1226 Draw
12:54:05: Game 1227 Draw
12:54:06: Game 1228 X wins!
12:54:06: Game 1229 Draw
12:54:06: Game 1230 Draw
12:54:06: Game 1231 Draw
12:54:06: Game 1232 Draw
12:54:06: Game 1233 Draw
12:54:07: Game 1234 X wins!
12:54:07: Game 1235 Draw
12:54:07: Game 1236 O wins!
12:54:07: Game 1237 Draw
12:54:07: Game 1238 Draw
12:54:08: Game 1239 Draw
12:54:08: Game 1240 Draw
12:54:08: Game 1241 Draw
12:54:08: Game 1242 Draw
12:54:08: Game 1243 Draw
12:54:08: Game 1244 Draw
12:54:09: Game 1245 Draw
12:54:09: Game 1246 Draw
12:54:09: Game 1247 X wins!
12:54:09: Game 1248 Draw
12:5

...

13:33:44: Game 8714 X wins!
13:33:44: Game 8715 Draw
13:33:45: Game 8716 Draw
13:33:45: Game 8717 Draw
13:33:46: Game 8718 Draw
13:33:46: Game 8719 Draw
13:33:46: Game 8720 Draw
13:33:47: Game 8721 Draw
13:33:47: Game 8722 Draw
13:33:48: Game 8723 Draw
13:33:48: Game 8724 Draw
13:33:49: Game 8725 Draw
13:33:49: Game 8726 Draw
13:33:50: Game 8727 Draw
13:33:50: Game 8728 Draw
13:33:51: Game 8729 Draw
13:33:51: Game 8730 Draw
13:33:51: Game 8731 Draw
13:33:52: Game 8732 Draw
13:33:52: Game 8733 Draw
13:33:53: Game 8734 Draw
13:33:53: Game 8735 Draw
13:33:54: Game 8736 Draw
13:33:54: Game 8737 Draw
13:33:55: Game 8738 Draw
13:33:55: Game 8739 Draw
13:33:56: Game 8740 Draw
13:33:56: Game 8741 Draw
13:33:57: Game 8742 Draw
13:33:57: Game 8743 Draw
13:33:57: Game 8744 Draw
13:33:58: Game 8745 Draw
13:33:58: Game 8746 Draw
13:33:59: Game 8747 Draw
13:33:59: Game 8748 Draw
13:34:00: Game 8749 Draw
13:34:00: Game 8750 Draw
13:34:01: Game 8751 Draw
13:34:01: Game 8752 Draw
13:34:02: Game 8753 Dr

13:36:07: Game 9029 Draw
13:36:07: Game 9030 Draw
13:36:08: Game 9031 Draw
13:36:08: Game 9032 Draw
13:36:08: Game 9033 Draw
13:36:09: Game 9034 Draw
13:36:09: Game 9035 Draw
13:36:10: Game 9036 Draw
13:36:10: Game 9037 Draw
13:36:11: Game 9038 Draw
13:36:11: Game 9039 Draw
13:36:12: Game 9040 Draw
13:36:12: Game 9041 Draw
13:36:13: Game 9042 Draw
13:36:13: Game 9043 Draw
13:36:13: Game 9044 Draw
13:36:14: Game 9045 X wins!
13:36:14: Game 9046 Draw
13:36:15: Game 9047 Draw
13:36:15: Game 9048 Draw
13:36:16: Game 9049 Draw
13:36:16: Game 9050 Draw
13:36:17: Game 9051 Draw
13:36:17: Game 9052 Draw
13:36:17: Game 9053 Draw
13:36:18: Game 9054 Draw
13:36:18: Game 9055 Draw
13:36:19: Game 9056 Draw
13:36:19: Game 9057 Draw
13:36:20: Game 9058 Draw
13:36:20: Game 9059 Draw
13:36:21: Game 9060 Draw
13:36:21: Game 9061 Draw
13:36:21: Game 9062 Draw
13:36:22: Game 9063 Draw
13:36:22: Game 9064 Draw
13:36:23: Game 9065 Draw
13:36:23: Game 9066 Draw
13:36:24: Game 9067 Draw
13:36:24: Game 9068 Dr

13:38:28: Game 9343 Draw
13:38:29: Game 9344 Draw
13:38:29: Game 9345 Draw
13:38:30: Game 9346 Draw
13:38:30: Game 9347 Draw
13:38:31: Game 9348 Draw
13:38:31: Game 9349 Draw
13:38:32: Game 9350 Draw
13:38:32: Game 9351 Draw
13:38:32: Game 9352 Draw
13:38:33: Game 9353 Draw
13:38:33: Game 9354 Draw
13:38:34: Game 9355 Draw
13:38:34: Game 9356 Draw
13:38:35: Game 9357 Draw
13:38:35: Game 9358 Draw
13:38:36: Game 9359 Draw
13:38:36: Game 9360 Draw
13:38:37: Game 9361 Draw
13:38:37: Game 9362 Draw
13:38:37: Game 9363 Draw
13:38:38: Game 9364 Draw
13:38:38: Game 9365 Draw
13:38:39: Game 9366 Draw
13:38:39: Game 9367 Draw
13:38:40: Game 9368 O wins!
13:38:40: Game 9369 Draw
13:38:41: Game 9370 Draw
13:38:41: Game 9371 Draw
13:38:42: Game 9372 Draw
13:38:42: Game 9373 Draw
13:38:42: Game 9374 Draw
13:38:43: Game 9375 Draw
13:38:43: Game 9376 Draw
13:38:44: Game 9377 Draw
13:38:44: Game 9378 Draw
13:38:45: Game 9379 Draw
13:38:45: Game 9380 Draw
13:38:46: Game 9381 Draw
13:38:46: Game 9382 Dr

13:40:56: Game 9657 Draw
13:40:57: Game 9658 Draw
13:40:57: Game 9659 Draw
13:40:57: Game 9660 Draw
13:40:58: Game 9661 Draw
13:40:58: Game 9662 Draw
13:40:59: Game 9663 Draw
13:40:59: Game 9664 Draw
13:41:00: Game 9665 Draw
13:41:00: Game 9666 Draw
13:41:01: Game 9667 Draw
13:41:01: Game 9668 Draw
13:41:02: Game 9669 Draw
13:41:02: Game 9670 Draw
13:41:03: Game 9671 Draw
13:41:03: Game 9672 Draw
13:41:04: Game 9673 Draw
13:41:04: Game 9674 Draw
13:41:05: Game 9675 Draw
13:41:05: Game 9676 Draw
13:41:05: Game 9677 Draw
13:41:06: Game 9678 Draw
13:41:06: Game 9679 Draw
13:41:07: Game 9680 Draw
13:41:07: Game 9681 Draw
13:41:08: Game 9682 Draw
13:41:08: Game 9683 Draw
13:41:09: Game 9684 Draw
13:41:09: Game 9685 Draw
13:41:10: Game 9686 Draw
13:41:10: Game 9687 Draw
13:41:11: Game 9688 Draw
13:41:11: Game 9689 Draw
13:41:11: Game 9690 Draw
13:41:12: Game 9691 Draw
13:41:12: Game 9692 Draw
13:41:13: Game 9693 Draw
13:41:13: Game 9694 Draw
13:41:14: Game 9695 Draw
13:41:14: Game 9696 Draw


13:43:25: Game 9971 Draw
13:43:25: Game 9972 Draw
13:43:25: Game 9973 Draw
13:43:26: Game 9974 Draw
13:43:26: Game 9975 Draw
13:43:27: Game 9976 Draw
13:43:27: Game 9977 Draw
13:43:28: Game 9978 Draw
13:43:28: Game 9979 Draw
13:43:29: Game 9980 Draw
13:43:29: Game 9981 Draw
13:43:30: Game 9982 Draw
13:43:30: Game 9983 Draw
13:43:31: Game 9984 Draw
13:43:31: Game 9985 Draw
13:43:32: Game 9986 Draw
13:43:32: Game 9987 Draw
13:43:33: Game 9988 Draw
13:43:33: Game 9989 Draw
13:43:34: Game 9990 Draw
13:43:34: Game 9991 Draw
13:43:35: Game 9992 Draw
13:43:35: Game 9993 Draw
13:43:35: Game 9994 Draw
13:43:36: Game 9995 Draw
13:43:36: Game 9996 Draw
13:43:37: Game 9997 Draw
13:43:37: Game 9998 Draw


In [18]:
V_model = load_model('model')

while True:
    g = Game()    
    playerX = HumanAgent(g, 'X')
    playerO = DeepRLagent(g, V_model, V_hist, 'O')
    
    max_moves = BOARD_SIZE * BOARD_SIZE
    winner = None
    
    for _ in range(max_moves):
        clear_output()
        print(g)
        
        player = g.player
        if player == 'X':
            row, col = playerX.get_move()
        else:
            move = playerO.select_move(verbose=False, exploration_rate=0)            
            g.play(move)

        if g.is_winner(player):
            winner = player
            break

    clear_output()
    print(g)
    if winner is None:
        print("Draw")
    else:
        print("%s wins!" % winner)

    if not play_again():
        print("Bye!")
        break


 O | X | X
 X | O | O
 X | O | X


Draw
Play again? (y or n)
n
Bye!
